# 1- Gdsfactory schematic
https://gdsfactory.github.io/gdsfactory/notebooks/10_schematic.html

it uses Networkx

❌ it needs manual position setting for each component

In [ ]:
import gdsfactory as gf
import gdsfactory.typings as gt

s = gt.Schematic()
s.add_instance("s11", gt.Instance(component=gf.c.mmi1x2()))
s.add_instance("s21", gt.Instance(component=gf.c.mmi1x2()))
s.add_instance("s22", gt.Instance(component=gf.c.mmi1x2()))
s.add_placement("s11", gt.Placement(x=0))
s.add_placement("s21", gt.Placement(x=10, y=+10))
s.add_placement("s22", gt.Placement(x=10, y=-10))
s.add_net(gt.Net(ip1="s11,o2", ip2="s21,o1"))
s.add_net(gt.Net(ip1="s11,o3", ip2="s22,o1"))
g = s.plot_netlist()

# 2- Gplugins schematic
https://gdsfactory.github.io/gplugins/notebooks/20_schematic_driven_layout.html

provides an interactive web interface. this can be used to drag components around to set their placement. 

❌ not abstract

In [ ]:
import gdsfactory as gf
from bokeh.io import output_notebook
from gplugins.schematic_editor import SchematicEditor

gf.config.rich_output()

%env BOKEH_ALLOW_WS_ORIGIN=*

output_notebook()

se = SchematicEditor("test.yml")

se.instance_widget

In [ ]:
se.instances.keys()

In [ ]:
se.instances["mmi1"].settings

In [ ]:
se.add_net(inst1="mmi1", port1="o2", inst2="s1", port2="o1")
se.add_net(inst1="mmi2", port1="o2", inst2="s1", port2="o2")
se.add_net(inst1="mmi1", port1="o3", inst2="s2", port2="o1")
se.add_net(inst1="mmi2", port1="o1", inst2="s2", port2="o2")
se.schematic

In [ ]:
se.visualize()

# 3- Networkx

In [ ]:
import networkx as nx

# Create a new graph
G = nx.Graph()

# Add nodes with attributes
G.add_node("Laser", function="Source")
G.add_node("BeamSplitter", function="Splitter")
G.add_node("Detector1", function="Detector")
G.add_node("Detector2", function="Detector")

# Connect nodes with edges
G.add_edge("Laser", "BeamSplitter")
G.add_edge("BeamSplitter", "Detector1")
G.add_edge("BeamSplitter", "Detector2")

# Draw the graph
pos = nx.spring_layout(G)  # positions for all nodes
nx.draw(
    G,
    pos,
    with_labels=True,
    node_size=2000,
    node_color="skyblue",
    font_size=10,
    font_weight="bold",
)

# labels = nx.get_node_attributes(G, 'function')
# nx.draw_networkx_labels(G, pos, labels=labels)

# Networkx and unconnected ports

we can add pseudo-nodes to visualize unconnected ports

In [ ]:
import networkx as nx


# Helper function to add pseudo-nodes for unconnected ports
def add_pseudo_nodes(graph):
    for node, data in list(graph.nodes(data=True)):
        if "total_ports" in data:  # Ensure we are dealing with main nodes
            total_ports = data["total_ports"]
            connected_ports = len(data["connected_ports"])
            for i in range(total_ports - connected_ports):
                pseudo_node = f"{node}_port{i+1}"
                graph.add_node(pseudo_node, pseudo=True)
                graph.add_edge(node, pseudo_node)


# Function to draw the graph
def draw_graph(graph):
    add_pseudo_nodes(graph)
    pos = nx.spring_layout(graph)  # Layout for visual consistency

    # Separate the nodes by type
    main_nodes = [
        node for node, attr in graph.nodes(data=True) if not attr.get("pseudo", False)
    ]
    pseudo_nodes = [
        node for node, attr in graph.nodes(data=True) if attr.get("pseudo", False)
    ]

    # Draw main nodes
    nx.draw_networkx_nodes(
        graph,
        pos,
        nodelist=main_nodes,
        node_size=2000,
        node_color="skyblue",
        label="Main Nodes",
    )
    # Draw pseudo-nodes
    nx.draw_networkx_nodes(
        graph,
        pos,
        nodelist=pseudo_nodes,
        node_size=1000,
        node_color="lightgrey",
        label="Pseudo Nodes",
    )

    # Draw edges
    nx.draw_networkx_edges(graph, pos)

    # Draw node labels
    labels = {node: node for node in main_nodes}
    nx.draw_networkx_labels(graph, pos, labels, font_size=12)

    # Show legend
    # plt.legend(scatterpoints=1)
    # plt.show()


# Sample usage with your defined graph
G = nx.Graph()

# Add nodes with a 'total_ports' attribute
nodes_with_ports = {"Laser": 2, "BeamSplitter": 3, "Detector1": 1, "Detector2": 1}

for node, ports in nodes_with_ports.items():
    G.add_node(node, total_ports=ports, connected_ports=[])

# Connect some nodes with edges and record the connections
connections = [
    ("Laser", "BeamSplitter"),
    ("BeamSplitter", "Detector1"),
    ("BeamSplitter", "Detector2"),
]

for src, dest in connections:
    G.nodes[src]["connected_ports"].append(dest)
    G.nodes[dest]["connected_ports"].append(src)
    G.add_edge(src, dest)

draw_graph(G)

# Customized Gdsfactory schematic

calling nx.spring_layout to automatically place the nodes.

In [ ]:
import CustomizedTypings as tt
import gdsfactory as gf

s = tt.Schematic()
s.add_instance("splitter1", tt.Instance(component=gf.c.mmi1x2()))
s.add_instance("wg1", tt.Instance(component=gf.c.straight()))
s.add_instance("splitter2", tt.Instance(component=gf.c.mmi1x2()))
s.add_net(tt.Net(ip1="splitter1,o2", ip2="wg1,o1"))
s.add_net(tt.Net(ip1="wg1,o2", ip2="splitter2,o2"))
s.add_net(tt.Net(ip1="splitter1,o3", ip2="splitter2,o3"))
g = s.plot_netlist()

# Graphviz

In [ ]:
import pygraphviz as pgv
from IPython.display import Image

# Create a directed graph
G = pgv.AGraph(strict=False, directed=False)

# Add nodes
G.add_node("Laser", shape="star")
G.add_node("BeamSplitter", shape="box")
G.add_node("Detector1", shape="invtriangle")
G.add_node("Detector2", shape="egg")

# Define ports with specific positions on the node's boundary
G.add_edge("Laser", "BeamSplitter", headport="n", tailport="n")
G.add_edge("BeamSplitter", "Detector1", headport="ne", tailport="e")
G.add_edge("BeamSplitter", "Detector2", headport="sw", tailport="w")

# Render the graph
G.draw("graph.png", format="png", prog="dot")
# G.write("file.dot")
print(G.string())
Image(filename="graph.png")